In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import re
from datetime import datetime 

In [2]:
AAPL = requests.get(url="https://seekingalpha.com/api/sa/combined/AAPL.xml")
TSLA = requests.get(url="https://seekingalpha.com/api/sa/combined/TSLA.xml")
GE = requests.get(url="https://seekingalpha.com/api/sa/combined/GE.xml") 
print(AAPL.status_code,TSLA.status_code,GE.status_code,)
print(AAPL.ok,TSLA.ok,GE.ok)

#requests package from apple, tesla and ge

200 200 200
True True True


In [3]:
aapl_root = ET.fromstring(AAPL.text)
tsla_root = ET.fromstring(TSLA.text)
ge_root = ET.fromstring(GE.text)
three_stocks=[aapl_root,tsla_root,ge_root] 

#Query their api and put three of them in a list for convenience 

In [4]:
df=pd.DataFrame(columns=('Symbol','Title','Author','Publication date','Company name'))
symbol=[]
title=[]
author=[]
pdate=[]
company=[]
df 

#make an empty dataframe with just column name.

,Symbol,Title,Author,Publication date,Company name


In [5]:
for root in three_stocks:
    for child in root:
        for grandchild in child:
            for i in grandchild:
                if re.search('title',i.tag):
                    title.append(i.text)
                elif re.search('author_name',i.tag):
                    author.append(i.text)
                elif re.search('pubDate',i.tag):
                    pdate.append(i.text)   

#look for correct tag and corresponding text(which is data I'm looking for)

In [6]:
df['Title']=title
df['Author']=author
df['Publication date']=pdate  

#add data into the dataframe

In [7]:
for i in range(90):
    if i < 30:
        df.loc[i]['Symbol']='AAPL'
        df.loc[i]['Company name']='Apple Inc.'
    elif i < 60:
        df.loc[i]['Symbol']='TSLA'
        df.loc[i]['Company name']='Tesla Motors'
    else:
        df.loc[i]['Symbol']='GE'
        df.loc[i]['Company name']='General Electric Company'

#since the limit of retriving data is 30 times, so 
#I'm pretty sure 0-29 is about apple, 29-59 is about tesla, and
#60-89 is about ge.

In [8]:
date_to_compare=[]
for i in pdate:
    date_to_compare.append(datetime.strptime(i[5:-6], '%d %b %Y %H:%M:%S'))
df['date_to_compare']=date_to_compare 

#Inorder to correctly sort the time, I need to convert date to a python recognizable format
#week and timezone is useless is this case so [5:-6] to get rid of them
#then add the comparable time as a new column into the dataframe

In [9]:
df=df.sort_values(by="date_to_compare",ascending=False) 
df

#Sort data by 'comparable' time in descending order

,Symbol,Title,Author,Publication date,Company name,date_to_compare
30,TSLA,Tesla: Deep Dive On The Solar Segment,ValueMan,"Fri, 15 Jan 2021 21:22:38 -0500",Tesla Motors,2021-01-15 21:22:38
31,TSLA,Tesla And Its Energy Business: A Deteriorating...,Bill Cunningham,"Fri, 15 Jan 2021 19:28:37 -0500",Tesla Motors,2021-01-15 19:28:37
0,AAPL,"Apple roundup: Re-closing some stores, plannin...",Jason Aycock,"Fri, 15 Jan 2021 18:24:30 -0500",Apple Inc.,2021-01-15 18:24:30
32,TSLA,Expect Positive Surprises For Tesla From Asia ...,Nick Cox,"Fri, 15 Jan 2021 12:15:32 -0500",Tesla Motors,2021-01-15 12:15:32
33,TSLA,The 3 Scenarios For Tesla's Value,Dimitrios Koutsoubos,"Fri, 15 Jan 2021 12:11:55 -0500",Tesla Motors,2021-01-15 12:11:55
...,...,...,...,...,...,...
85,GE,GE aiming to eliminate more jobs in aviation s...,Carl Surran,"Tue, 24 Nov 2020 09:12:02 -0500",General Electric Company,2020-11-24 09:12:02
86,GE,"GE rallies 3% for second day, this time after ...",Yoel Minkoff,"Tue, 24 Nov 2020 07:36:57 -0500",General Electric Company,2020-11-24 07:36:57
87,GE,General Electric to develop LNG power plant in...,Yoel Minkoff,"Mon, 23 Nov 2020 07:36:48 -0500",General Electric Company,2020-11-23 07:36:48
88,GE,U.S. to ban tech exports to 89 firms in latest...,Yoel Minkoff,"Mon, 23 Nov 2020 04:05:46 -0500",General Electric Company,2020-11-23 04:05:46


In [10]:
df=df.drop(columns='date_to_compare')

#at the last, drop the extra column. keep the order.

In [11]:
df[0:45]

,Symbol,Title,Author,Publication date,Company name
30,TSLA,Tesla: Deep Dive On The Solar Segment,ValueMan,"Fri, 15 Jan 2021 21:22:38 -0500",Tesla Motors
31,TSLA,Tesla And Its Energy Business: A Deteriorating...,Bill Cunningham,"Fri, 15 Jan 2021 19:28:37 -0500",Tesla Motors
0,AAPL,"Apple roundup: Re-closing some stores, plannin...",Jason Aycock,"Fri, 15 Jan 2021 18:24:30 -0500",Apple Inc.
32,TSLA,Expect Positive Surprises For Tesla From Asia ...,Nick Cox,"Fri, 15 Jan 2021 12:15:32 -0500",Tesla Motors
33,TSLA,The 3 Scenarios For Tesla's Value,Dimitrios Koutsoubos,"Fri, 15 Jan 2021 12:11:55 -0500",Tesla Motors
34,TSLA,Tesla forecast to deliver one million cars in ...,Clark Schultz,"Fri, 15 Jan 2021 07:15:00 -0500",Tesla Motors
1,AAPL,Xiaomi plunges 10% as U.S. adds smartphone gia...,Yoel Minkoff,"Fri, 15 Jan 2021 05:45:41 -0500",Apple Inc.
60,GE,GE accuses Siemens Energy in lawsuit of steali...,Carl Surran,"Thu, 14 Jan 2021 15:19:56 -0500",General Electric Company
35,TSLA,Tesla's Model Y earns top scores from NHTSA,Clark Schultz,"Wed, 13 Jan 2021 15:39:34 -0500",Tesla Motors
36,TSLA,Tesla: The 2021 Growth Surge,Bill Maurer,"Wed, 13 Jan 2021 13:38:54 -0500",Tesla Motors


In [12]:
df[45:90]

,Symbol,Title,Author,Publication date,Company name
52,TSLA,Tesla nabs Street-high $810 PT at Morgan Stanl...,Carl Surran,"Tue, 05 Jan 2021 17:12:34 -0500",Tesla Motors
65,GE,GE will not claw back Immelt's pay - WSJ,Carl Surran,"Tue, 05 Jan 2021 16:49:18 -0500",General Electric Company
53,TSLA,"Bill Gross sees SPACs, growth stocks strugglin...",Liz Kiesche,"Tue, 05 Jan 2021 13:44:12 -0500",Tesla Motors
18,AAPL,Apple lands higher estimates at Canaccord due ...,Clark Schultz,"Tue, 05 Jan 2021 13:07:51 -0500",Apple Inc.
19,AAPL,Apple: Waning Momentum Meets Extreme Valuation...,Stuart Allsopp,"Tue, 05 Jan 2021 12:27:46 -0500",Apple Inc.
54,TSLA,Tesla seen by Exane shedding 50% in tech titan...,Clark Schultz,"Tue, 05 Jan 2021 10:33:35 -0500",Tesla Motors
20,AAPL,"Apple: Autonomous Driving, Services And 5G Cou...",Steven Fiorillo,"Tue, 05 Jan 2021 09:30:00 -0500",Apple Inc.
55,TSLA,"A Bubble-Blowing, Bull Market, Blow-Off Top Ph...",Alpha Trader,"Tue, 05 Jan 2021 08:00:00 -0500",Tesla Motors
56,TSLA,Tesla Falls Short But Shares Surge,Bill Maurer,"Mon, 04 Jan 2021 20:07:42 -0500",Tesla Motors
57,TSLA,"Byron Wien's ""10 Surprises also rans"" include ...",David Jackson,"Mon, 04 Jan 2021 14:49:39 -0500",Tesla Motors
